In [70]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks


In [71]:
for dirname, _, filenames in os.walk('/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [72]:
## load data
data_path = {
    'categories': '../data/item_categories.csv',
    'items': '../data/items.csv',
    'sales': '../data/sales_train.csv',
    'shops': '../data/shops.csv',
    'test': '../data/test.csv'
}
print('dataset loaded completly')


dataset loaded completly


In [73]:
dataset = { name: pd.read_csv(path) for name, path in data_path.items()}
dataset.keys()

dict_keys(['categories', 'items', 'sales', 'shops', 'test'])

Inspect data

In [74]:
def inspect_data(dataset: pd.DataFrame):
    print(dataset.info())

    print(f"""
        Inspecting for duplicates:
        Discovered: {sum(dataset.value_counts() > 1)}
    """)

    return dataset.head()

In [75]:
inspect_data(dataset['sales'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB
None

        Inspecting for duplicates:
        Discovered: 6
    


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [76]:
inspect_data(dataset['items'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB
None

        Inspecting for duplicates:
        Discovered: 0
    


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [77]:
inspect_data(dataset['shops'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   shop_name  60 non-null     object
 1   shop_id    60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB
None

        Inspecting for duplicates:
        Discovered: 0
    


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [78]:
duplications = dataset['sales'].value_counts()
duplications

date        date_block_num  shop_id  item_id  item_price  item_cnt_day
23.03.2014  14              21       3423     999.0       1.0             2
01.05.2014  16              50       3423     999.0       1.0             2
23.02.2014  13              50       3423     999.0       1.0             2
12.07.2014  18              25       3423     999.0       1.0             2
31.12.2014  23              42       21619    499.0       1.0             2
                                                                         ..
21.08.2013  7               44       13344    25.0        1.0             1
                                     12817    149.0       1.0             1
                                     12665    399.0       1.0             1
                                     12181    429.0       1.0             1
01.01.2013  0               2        991      99.0        1.0             1
Length: 2935843, dtype: int64

In [63]:
duplications = duplications[duplications > 1]
duplications

date        date_block_num  shop_id  item_id  item_price  item_cnt_day
23.03.2014  14              21       3423     999.0       1.0             2
01.05.2014  16              50       3423     999.0       1.0             2
23.02.2014  13              50       3423     999.0       1.0             2
12.07.2014  18              25       3423     999.0       1.0             2
31.12.2014  23              42       21619    499.0       1.0             2
05.01.2013  0               54       20130    149.0       1.0             2
dtype: int64

In [79]:
inspect_data(dataset['test'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   ID       214200 non-null  int64
 1   shop_id  214200 non-null  int64
 2   item_id  214200 non-null  int64
dtypes: int64(3)
memory usage: 4.9 MB
None

        Inspecting for duplicates:
        Discovered: 0
    


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [80]:
train = pd.read_csv(data_path['sales'])
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [86]:
train['date'] = pd.to_datetime(train['date'])
train.head()

,date,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,59,22154,999.00,1.0
1,2013-03-01,25,2552,899.00,1.0
2,2013-05-01,25,2552,899.00,-1.0
3,2013-06-01,25,2554,1709.05,1.0
4,2013-01-15,25,2555,1099.00,1.0


In [93]:
train['date'] = train['date'].apply(lambda x: x.strftime("%Y-%m"))


In [94]:
test = pd.read_csv(data_path['test'])
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [95]:
comp_data = train.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
comp_data.reset_index(inplace=True)
comp_data.head()

date,shop_id,item_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,...,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12
0,0,30,0.000000,3.444444,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,31,0.000000,1.571429,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,32,1.333333,1.500000,2.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,33,1.000000,1.000000,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,35,1.000000,1.200000,0.0,1.0,0.0,2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
test_comp = pd.merge(test, comp_data, on=['shop_id','item_id'], how='left')
test_comp = test_comp.fillna(0)


In [97]:
test_comp.head()

,ID,shop_id,item_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,...,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.5,1.0,0.0,1.0,1.0,0.0,0.0,0.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
